# Global Contrast Normalization and ZCA whitening

### GCA

In [87]:
# Code base from
# https://datascience.stackexchange.com/questions/15110/how-to-implement-global-contrast-normalization-in-python
# and from
# https://github.com/yumatsuoka/zca_exercize/blob/master/src/appZCAtoCIFAR10.py

import numpy as np
import scipy
import scipy.misc
from PIL import Image
from keras.datasets import cifar10, cifar100


def global_contrast_normalization(X, s = 1, lmda = 10, epsilon = 1e-9):
    
    # replacement for the loop
    X_average = np.mean(X)
    X = X - X_average

    # `su` is here the mean, instead of the sum
    contrast = np.sqrt(lmda + np.mean(X**2))
    X = s * X / max(contrast, epsilon)
    
    return X

def load_and_gbc_cifar(classes = 10):
    
    if classes == 10:
        (X_train, Y_train), (X_test, y_test) = cifar10.load_data()
    else:
        (X_train, Y_train), (X_test, y_test) = cifar100.load_data()
        
    
    X_train_gbc = []
    X_test_gbc = []
    
    # Image by image global contrast normalization
    for img in X_train:
        X_train_gbc.append(global_contrast_normalization(img))
        
    for img in X_test:
        X_test_gbc.append(global_contrast_normalization(img))
        
    X_train_gbc = np.array(X_train_gbc)
    X_test_gbc = np.array(X_test_gbc)
        
    return (X_train_gbc, Y_train), (X_test_gbc, y_test)

In [89]:
from keras.datasets import cifar10
(X_train, Y_train), (X_test, y_test) = load_and_gbc_cifar(classes=10)

print(X_train.shape)
X_test.shape

(50000, 32, 32, 3)


(10000, 32, 32, 3)

In [90]:
# PCA fitting
def principal_components(x):
    x = x.transpose(0, 2, 3, 1)
    flatx = numpy.reshape(x, (x.shape[0], x.shape[1] * x.shape[2] * x.shape[3]))  # Flatten image
    sigma = numpy.dot(flatx.T, flatx) / flatx.shape[1]  # Calculate sigma
    U, S, V = numpy.linalg.svd(sigma)  # Singular value decomposition
    eps = 0.0001
    return numpy.dot(numpy.dot(U, numpy.diag(1. / numpy.sqrt(S + eps))), U.T)


print("PCA fitting")
pc = principal_components(X_train)
print("PCA done")

PCA fitting
PCA done


In [126]:
# ZCA whitening
def zca_whitening(x, principal_components):
    print(x.shape)
    #x = x.transpose(1,2,0)
    #print(x.shape)
    flatx = numpy.reshape(x, (x.size))
    whitex = numpy.dot(flatx, principal_components)
    x = numpy.reshape(whitex, (x.shape[0], x.shape[1], x.shape[2]))
    return x

In [132]:
test_img = zca_whitening(X_train[3], pc)

(32, 32, 3)


In [133]:
test_img.shape

(32, 32, 3)

In [134]:
scipy.misc.imsave("test.jpg", test_img)